purpose: check which network props failed for tissue specific networks and figure out why

In [1]:
import pandas as pd
import sys
import os
from network_functions import *
import re

In [2]:
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/')

In [3]:
outdir='tissue_networks/intermediate/'

In [4]:
files=[x for x in os.listdir('network_scores') if 'tissue' in x]

In [5]:
tissues = ["basal_ganglion", "brain", "caudate_nucleus", "caudate_putamen", 
    "central_nervous_system", "cerebellar_cortex", "cerebellum", 
    "cerebral_cortex", "corpus_callosum", "corpus_striatum", 
    "dentate_gyrus", "diencephalon", "forebrain", "frontal_lobe", 
    "glia", "hippocampus", "hypophysis", "hypothalamus", 
    "locus_ceruleus", "medulla_oblongata", "midbrain", "nervous_system", 
    "neuron", "nucleus_accumbens", "occipital_lobe", "occipital_pole", 
    "pons", "peripheral_nervous_system", "spinal_cord", 
    "substantia_nigra", "subthalamic_nucleus", "telencephalon", 
    "temporal_lobe", "thalamus", "bronchial_epithelial_cell", "bronchus", 
    "lung", "trachea", "global"]

In [6]:
f= [re.findall(r"(.*?)_hb_tissue_(.*?)_top_zscore\.tsv", x)[0] for x in files]

In [7]:
tissue_network=True
tissue='global'
netdir='tissue_networks/intermediate/'

print(f'available files for this tissue: {[x for x in os.listdir(netdir) if tissue in x]}')
interactome_name=f'hb_tissue_{tissue}_top'
#import node list
with open(f'{netdir}node_list_{tissue}_top.txt', 'r') as file:
    lines = file.readlines()
all_nodes=[line.strip() for line in lines]

available files for this tissue: ['node_list_global_top.txt', 'w_double_prime_global.npy', 'degree_global.csv', 'node_list_global.txt', 'w_double_prime_global_top.npy', 'normalized_adjacency_global_top.npz', 'degree_global_top.csv', 'normalized_adjacency_global.npz']


In [15]:
hgnc

,symbol,entrez_id
0,A1BG,1
1,A1BG-AS1,503538
2,A1CF,29974
3,A2M,2
4,A2M-AS1,144571
...,...,...
44062,FADS2B,643181
44063,FADS3,3995
44064,FADS6,283985
44065,FAF1,11124


In [16]:
seed_count={}

In [17]:
hgnc=pd.read_csv('hgnc_complete_set.txt',sep='\t',low_memory=False)
hgnc=hgnc[['symbol','entrez_id']].dropna()
hgnc['entrez_id']=hgnc['entrez_id'].astype(int).astype(str)
ctrl_traits=['facial_hair', 'age_smkinit', 'antisoc', 'friend_sat', 'hr', 'infant_bw', 'LDL', 'maternal_smok', 'townsend', 'age_menarche', 'neurot','addict-rf']
for tissue in tissues:
    interactome_name=f'hb_tissue_{tissue}_top'
    #import node list
    with open(f'{netdir}node_list_{tissue}_top.txt', 'r') as file:
        lines = file.readlines()
    all_nodes=[line.strip() for line in lines]
    
    seed_dict=import_seed_dict(mag_dir,file_dict,bonf_dict,gene_col_dict,hgnc[hgnc.entrez_id.isin(all_nodes)]['symbol']) 
    ctrl_dict={}
    for t in ctrl_traits:
        ctrl_dict[t]=pd.read_csv('gwas_ctrl_hm/magma/seed_genes/'+t+'_annot.tsv',sep='\t')
    for t in ctrl_traits:
        seed_dict[t+'_FDR']=(set(ctrl_dict[t][ctrl_dict[t]['Q']<0.05]['GENE']))
        seed_dict[t+'_bonf']=(set(ctrl_dict[t][ctrl_dict[t]['P']<0.05/len(ctrl_dict[t])]['GENE']))
        if not tissue_network:
            seed_dict[t+'_top500']=set(ctrl_dict[t][(ctrl_dict[t]['GENE'].isin(all_nodes))].nsmallest(500,'P')['GENE'])
        else:
            seed_dict[t+'_top500']=set(ctrl_dict[t][(ctrl_dict[t]['GENE'].isin(hgnc[hgnc.entrez_id.isin(all_nodes)]['symbol']))].nsmallest(500,'P')['GENE'])
    if tissue_network:
        seed_dict={k: len(set(hgnc[hgnc.symbol.isin(v)]['entrez_id'])) for k, v in seed_dict.items()}
    
    print(tissue)
    seed_count[tissue]=(seed_dict)

basal_ganglion
brain
caudate_nucleus
caudate_putamen
central_nervous_system
cerebellar_cortex
cerebellum
cerebral_cortex
corpus_callosum
corpus_striatum
dentate_gyrus
diencephalon
forebrain
frontal_lobe
glia
hippocampus
hypophysis
hypothalamus
locus_ceruleus
medulla_oblongata
midbrain
nervous_system
neuron
nucleus_accumbens
occipital_lobe
occipital_pole
pons
peripheral_nervous_system
spinal_cord
substantia_nigra
subthalamic_nucleus
telencephalon
temporal_lobe
thalamus
bronchial_epithelial_cell
bronchus
lung
trachea
global


In [20]:
seed_count=pd.DataFrame(seed_count)

In [21]:
seed_count.to_csv('seed_gene_count.tsv',sep='\t')

In [67]:
if not tissue_network:
    seed_dict=import_seed_dict(mag_dir,file_dict,bonf_dict,gene_col_dict,all_nodes)
else:
    hgnc=pd.read_csv('hgnc_complete_set.txt',sep='\t',low_memory=False)
    hgnc=hgnc[['symbol','entrez_id']].dropna()
    hgnc['entrez_id']=hgnc['entrez_id'].astype(int).astype(str)
    seed_dict=import_seed_dict(mag_dir,file_dict,bonf_dict,gene_col_dict,hgnc[hgnc.entrez_id.isin(all_nodes)]['symbol']) 
seed_dict.keys()
#dictionary of human control traits
ctrl_dict={}
ctrl_traits=['facial_hair', 'age_smkinit', 'antisoc', 'friend_sat', 'hr', 'infant_bw', 'LDL', 'maternal_smok', 'townsend', 'age_menarche', 'neurot','addict-rf']
for t in ctrl_traits:
    ctrl_dict[t]=pd.read_csv('gwas_ctrl_hm/magma/seed_genes/'+t+'_annot.tsv',sep='\t')
for t in ctrl_traits:
    seed_dict[t+'_FDR']=(set(ctrl_dict[t][ctrl_dict[t]['Q']<0.05]['GENE']))
    seed_dict[t+'_bonf']=(set(ctrl_dict[t][ctrl_dict[t]['P']<0.05/len(ctrl_dict[t])]['GENE']))
    if not tissue_network:
        seed_dict[t+'_top500']=set(ctrl_dict[t][(ctrl_dict[t]['GENE'].isin(all_nodes))].nsmallest(500,'P')['GENE'])
    else:
        seed_dict[t+'_top500']=set(ctrl_dict[t][(ctrl_dict[t]['GENE'].isin(hgnc[hgnc.entrez_id.isin(all_nodes)]['symbol']))].nsmallest(500,'P')['GENE'])
if tissue_network:
    seed_dict={k: set(hgnc[hgnc.symbol.isin(v)]['entrez_id']) for k, v in seed_dict.items()}

In [48]:
seeds=set(seed_dict.keys())

In [50]:
missing_dict={}
for t in tissues:
    scores=[x for x in f if t in x]
    if len(scores)==len(seeds):
        print(f'all scores calculated for {t}')
    else:
        print(f'{len(seeds)-len(scores)} MISSING VALUES for {t}')
        missing_dict[t]=set(seed_dict).difference([x[0] for x in scores])

70 MISSING VALUES for basal_ganglion
18 MISSING VALUES for brain
18 MISSING VALUES for caudate_nucleus
18 MISSING VALUES for caudate_putamen
18 MISSING VALUES for central_nervous_system
18 MISSING VALUES for cerebellar_cortex
30 MISSING VALUES for cerebellum
21 MISSING VALUES for cerebral_cortex
18 MISSING VALUES for corpus_callosum
18 MISSING VALUES for corpus_striatum
18 MISSING VALUES for dentate_gyrus
18 MISSING VALUES for diencephalon
18 MISSING VALUES for forebrain
18 MISSING VALUES for frontal_lobe
18 MISSING VALUES for glia
18 MISSING VALUES for hippocampus
18 MISSING VALUES for hypophysis
22 MISSING VALUES for hypothalamus
22 MISSING VALUES for locus_ceruleus
18 MISSING VALUES for medulla_oblongata
18 MISSING VALUES for midbrain
18 MISSING VALUES for nervous_system
18 MISSING VALUES for neuron
22 MISSING VALUES for nucleus_accumbens
32 MISSING VALUES for occipital_lobe
32 MISSING VALUES for occipital_pole
32 MISSING VALUES for pons
18 MISSING VALUES for peripheral_nervous_syst

In [52]:
missing_dict.keys()

dict_keys(['basal_ganglion', 'brain', 'caudate_nucleus', 'caudate_putamen', 'central_nervous_system', 'cerebellar_cortex', 'cerebellum', 'cerebral_cortex', 'corpus_callosum', 'corpus_striatum', 'dentate_gyrus', 'diencephalon', 'forebrain', 'frontal_lobe', 'glia', 'hippocampus', 'hypophysis', 'hypothalamus', 'locus_ceruleus', 'medulla_oblongata', 'midbrain', 'nervous_system', 'neuron', 'nucleus_accumbens', 'occipital_lobe', 'occipital_pole', 'pons', 'peripheral_nervous_system', 'spinal_cord', 'substantia_nigra', 'subthalamic_nucleus', 'telencephalon', 'temporal_lobe', 'thalamus', 'bronchial_epithelial_cell', 'bronchus', 'lung', 'trachea'])

In [ ]:
ctrl_traits=['facial_hair', 'age_smkinit', 'antisoc', 'friend_sat', 'hr', 'infant_bw', 'LDL', 'maternal_smok', 'townsend', 'age_menarche', 'neurot','addict-rf']


In [70]:
missing_dict['cerebellum']

{'LDL_FDR',
 'LDL_bonf',
 'LDL_top500',
 'addict-rf_FDR',
 'addict-rf_bonf',
 'addict-rf_top500',
 'age_menarche_FDR',
 'age_menarche_bonf',
 'age_menarche_top500',
 'age_smkinit_top500',
 'antisoc_FDR',
 'antisoc_bonf',
 'antisoc_top500',
 'facial_hair_top500',
 'friend_sat_top500',
 'hr_top500',
 'infant_bw_FDR',
 'infant_bw_bonf',
 'infant_bw_top500',
 'loco_final_cf_rat_FDR',
 'loco_final_cf_rat_bonf',
 'maternal_smok_FDR',
 'maternal_smok_bonf',
 'maternal_smok_top500',
 'neurot_FDR',
 'neurot_bonf',
 'neurot_top500',
 'townsend_FDR',
 'townsend_bonf',
 'townsend_top500'}